## Week 4 Exercise 1 

In [1]:
from collections import Counter

In [2]:
test = 'the wheels on the bus go round and round'
Counter(test.split()).most_common()

[('round', 2),
 ('the', 2),
 ('and', 1),
 ('on', 1),
 ('bus', 1),
 ('go', 1),
 ('wheels', 1)]

# Documents Similarity

In [3]:
import graphlab as gl

In [4]:
people = gl.SFrame('Data/people_wiki.gl/')
people

This non-commercial license of GraphLab Create for academic use is assigned to huangchengyu16@gmail.com and will expire on September 03, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1473689120.log


URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


## 1. Explore the data set

In [5]:
## take a look at obama
obama = people[people['name']=='Barack Obama']
print obama,'\n'
print obama['text']

+-------------------------------+--------------+-------------------------------+
|              URI              |     name     |              text             |
+-------------------------------+--------------+-------------------------------+
| <http://dbpedia.org/resour... | Barack Obama | barack hussein obama ii br... |
+-------------------------------+--------------+-------------------------------+
[? rows x 3 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization. 

['barack hussein obama ii brk husen bm born august 4 1961 is the 44th and current president of the united states and the first african american to hold the office born in honolulu hawaii obama is a graduate of columbia university and harvard law school where he served as president of the harvard law review he was a community organizer in chicago before earning his law degree he worked as a civil rights attorney and taught constitution

In [6]:
import operator
## create word count
obama['word_count']= gl.text_analytics.count_words(obama['text'])

## 2. How to sort word count

In [7]:
# one way to sort 
## sort obama word count by value 
word_freq = sorted(obama['word_count'][0].items(), key=operator.itemgetter(1), reverse=True)
word_freq

[('the', 40),
 ('in', 30),
 ('and', 21),
 ('of', 18),
 ('to', 14),
 ('his', 11),
 ('obama', 9),
 ('act', 8),
 ('he', 7),
 ('a', 7),
 ('law', 6),
 ('us', 6),
 ('as', 6),
 ('was', 5),
 ('has', 4),
 ('for', 4),
 ('military', 4),
 ('iraq', 4),
 ('control', 4),
 ('after', 4),
 ('president', 4),
 ('democratic', 4),
 ('2011', 3),
 ('term', 3),
 ('january', 3),
 ('involvement', 3),
 ('response', 3),
 ('school', 3),
 ('senate', 3),
 ('election', 3),
 ('signed', 3),
 ('american', 3),
 ('first', 3),
 ('campaign', 3),
 ('from', 3),
 ('with', 3),
 ('ordered', 3),
 ('states', 3),
 ('united', 3),
 ('party', 3),
 ('2009', 3),
 ('2004', 3),
 ('office', 2),
 ('is', 2),
 ('domestic', 2),
 ('20', 2),
 ('policy', 2),
 ('2010', 2),
 ('during', 2),
 ('presidential', 2),
 ('university', 2),
 ('republican', 2),
 ('house', 2),
 ('national', 2),
 ('born', 2),
 ('second', 2),
 ('afghanistan', 2),
 ('representatives', 2),
 ('protection', 2),
 ('november', 2),
 ('on', 2),
 ('foreign', 2),
 ('harvard', 2),
 ('primar

In [8]:
## a second way to sroet 
# first made the dictionray into a two table
obama_word_count_table = obama[['word_count']].stack('word_count',new_column_name=['word','count'])
# sort it 
obama_word_count_table.sort('count',ascending=False)

word,count
the,40
in,30
and,21
of,18
to,14
his,11
obama,9
act,8
he,7
a,7


## 3. Compute TF-IDF for the corpus 

In [9]:
people['word_count']= gl.text_analytics.count_words(people['text'])
people.head()

URI,name,text,word_count
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'selection': 1,'carltons': 1, 'being': ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'precise': 1, 'thomas':1, 'closely': 1, ..."
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...,"{'just': 1, 'issued': 1,'mainly': 1, 'nominat ..."
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...,"{'all': 1,'bauforschung': 1, ..."
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...,"{'they': 1,'gangstergenka': 1, ..."
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...,"{'currently': 1, 'less':1, 'being': 1, ..."
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...,"{'exclusive': 2,'producer': 1, 'show' ..."
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...,"{'taxi': 1, 'salon': 1,'gangs': 1, 'being': 1, ..."
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...,"{'houston': 1, 'frankie':1, 'labels': 1, ..."
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...,"{'phenomenon': 1,'deborash': 1, 'both' ..."


In [10]:
## use the build function in text_analytics 
tfidf = gl.text_analytics.tf_idf(people['word_count'])

In [11]:
people['tfidf']=tfidf
people.head()

URI,name,text,word_count
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'selection': 1,'carltons': 1, 'being': ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'precise': 1, 'thomas':1, 'closely': 1, ..."
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...,"{'just': 1, 'issued': 1,'mainly': 1, 'nominat ..."
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...,"{'all': 1,'bauforschung': 1, ..."
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...,"{'they': 1,'gangstergenka': 1, ..."
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...,"{'currently': 1, 'less':1, 'being': 1, ..."
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...,"{'exclusive': 2,'producer': 1, 'show' ..."
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...,"{'taxi': 1, 'salon': 1,'gangs': 1, 'being': 1, ..."
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...,"{'houston': 1, 'frankie':1, 'labels': 1, ..."
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...,"{'phenomenon': 1,'deborash': 1, 'both' ..."


## 4. Examine tf-idf 

In [12]:
## take a look at obama
obama = people[people['name']=='Barack Obama']
obama[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
obama,43.2956530721
act,27.678222623
iraq,17.747378588
control,14.8870608452
law,14.7229357618
ordered,14.5333739509
military,13.1159327785
involvement,12.7843852412
response,12.7843852412
democratic,12.4106886973


## 5. Manually compute distances between a few people

In [13]:
clinton = people[people['name']=='Bill Clinton']
backham = people[people['name']=='David Beckham']

#### Is obama closer to clinton or backham

In [14]:
print 'Obama - Clinton distance: ',gl.distances.cosine(obama['tfidf'][0],clinton['tfidf'][0])
print 'Obama - Backham distance: ',gl.distances.cosine(obama['tfidf'][0],backham['tfidf'][0])
print 'Obama is closer to Clinton'

Obama - Clinton distance:  0.833985493688
Obama - Backham distance:  0.979130584475
Obama is closer to Clinton


## 6. Build a nearest neighbor model for document retrieval 

In [15]:
## create the nearest neighbor model 
knn_model = gl.nearest_neighbors.create(people,features=['tfidf'],label='name')

Starting brute force nearest neighbors model training.

## 7. Applying the nearest neighbor model for retrieval 

#### Who is closest to Obama?

In [16]:
knn_model.query(obama)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 29.177ms     |

| 0            | 20962   | 35.4861     | 1.14s        |

| 0            | 42513   | 71.9693     | 2.14s        |

| Done         |         | 100         | 2.96s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Barack Obama,0.0,1
0,Joe Biden,0.794117647059,2
0,Joe Lieberman,0.794685990338,3
0,Kelly Ayotte,0.811989100817,4
0,Bill Clinton,0.813852813853,5


#### Other examples of document retrival 

In [17]:
swift = people[people['name']== 'Taylor Swift']
knn_model.query(swift)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 10.266ms     |

| 0            | 32038   | 54.2364     | 1.01s        |

| 0            | 59018   | 99.9103     | 2.01s        |

| Done         |         | 100         | 2.01s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Taylor Swift,0.0,1
0,Carrie Underwood,0.76231884058,2
0,Alicia Keys,0.764705882353,3
0,Jordin Sparks,0.769633507853,4
0,Leona Lewis,0.776119402985,5


In [18]:
jolie = people[people['name']== 'Angelina Jolie']
knn_model.query(jolie)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 11.059ms     |

| 0            | 30562   | 51.7377     | 1.15s        |

| 0            | 55867   | 94.576      | 2.15s        |

| Done         |         | 100         | 2.20s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Angelina Jolie,0.0,1
0,Brad Pitt,0.784023668639,2
0,Julianne Moore,0.795857988166,3
0,Billy Bob Thornton,0.803069053708,4
0,George Clooney,0.8046875,5


In [19]:
arnold = people[people['name']== 'Arnold Schwarzenegger']
knn_model.query(arnold)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 10.933ms     |

| 0            | 30636   | 51.863      | 1.01s        |

| 0            | 51332   | 86.8988     | 2.01s        |

| Done         |         | 100         | 2.61s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Arnold Schwarzenegger,0.0,1
0,Jesse Ventura,0.818918918919,2
0,John Kitzhaber,0.824615384615,3
0,Lincoln Chafee,0.833876221498,4
0,Anthony Foxx,0.833910034602,5


In [21]:
### clear memory 
arnold = None
jolie = None
swift = None
clinton = None
backham = None
knn_model = None


# Exercise

### 1. Compare top words according to word counts to TF-IDF

In [20]:
## look a the person Elton John
john = people[people['name']== 'Elton John']

In [21]:
## top three words for him 
john[['word_count']].stack('word_count',new_column_name=['word','count']).sort('count',ascending=False)

word,count
the,27
in,18
and,15
of,13
a,10
has,9
john,7
he,7
on,6
award,5


In [22]:
## top three words by tfidf
john[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
tonightcandle,10.9864953892
overallelton,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


### 2. Measuring distance 

In [23]:
v_beckham = people[people['name']== 'Victoria Beckham']
mccartney = people[people['name']== 'Paul McCartney']

In [ ]:
print 'john - victoria beckham distance: ',gl.distances.cosine(john['tfidf'][0],v_beckham['tfidf'][0])
print 'john - mccartney distance: ',gl.distances.cosine(john['tfidf'][0],mccartney['tfidf'][0])
print 'John is closer to McCartney'

john - victoria beckham distance:  0.956700637666
john - mccartney distance:  0.825031002922
John is closer to McCartney


### 3.Building nearest neighbors models with different input features and distance metric

In [ ]:
knn_model_word_count = gl.nearest_neighbors.create(people,features=['word_count'],label='name',distance='cosine')

Starting brute force nearest neighbors model training.

In [ ]:
knn_model_tfidf = gl.nearest_neighbors.create(people,features=['tfidf'],label='name',distance='cosine')

Starting brute force nearest neighbors model training.

In [ ]:
## compare two 
print knn_model_word_count.query(john)
print knn_model_tfidf.query(john)

something
